In [1]:
# load packages

import time

from IPython import get_ipython
from IPython.display import clear_output
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csc_matrix
from scipy import signal as sg
import scipy
import pickle

from tifffile.tifffile import imwrite,imread
from tqdm.auto import tqdm,trange

from copy import deepcopy
import h5py

import caiman as cm
from caiman.source_extraction.cnmf import cnmf,params
from caiman.paths import caiman_datadir
from caiman.utils.visualization import get_contours

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

def load_pickle(file_path):
    """
    Load a dictionary from a pickle file.

    Args:
    - file_path (str): Path to the pickle file.

    Returns:
    - dict: Loaded dictionary.
    """
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

def flatten_extend(matrix):
     flat_list = []
     for row in matrix:
         flat_list.extend(row)
     return flat_list

In [2]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\nasquatch\data\2p\peter\in_vivo\220406_PK24PL10\wells101\preprocessing\220406_PK24PL10_wells101_jordan_done_qc_doneqc']
mmap_dir_l = [r'']
mov_means_l = [52]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [17]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\NASQUATCH\data\2p\peter\slice\20230824_SL1PL60\slice\preprocessing\20230824_SL1PL60_slice_charlotte_done_qc_doneqc',
             r'\\NASQUATCH\data\2p\peter\slice\20230824_SL1PL60\wells001\preprocessing\20230824_SL1PL60_wells001_charlotte_done_qc_doneqc']
             #r'\\NASQUATCH\data\2p\peter\slice\20230824_SL1PL60\wells101\preprocessing\20230824_SL1PL60_wells101_charlotte_done_qc_doneqc',
             #r'\\NASQUATCH\data\2p\peter\in_vivo\231107_DG44PL58\mouse\preprocessing\231107_DG44PL58_mouse_maddy_done_qc_done',
             #r'\\NASQUATCH\data\2p\peter\in_vivo\231107_DG44PL58\wells\preprocessing\231107_DG44PL58_wells_david_done_qc_doneqc',
             #r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells101\preprocessing\20230922_PL28_wells101_david_done_qc_doneqc',
             #r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells301\preprocessing\20230922_PL28_wells301_david_done_qc_doneqc',
             #r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells501\preprocessing\20230922_PL28_wells501_david_done_qc_doneqc',
             #r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells601\preprocessing\20230922_PL28_wells601_david_done_qc_doneqc',
             #r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells701\preprocessing\20230922_PL28_wells701_david_done_qc_doneqc']
mmap_dir_l = [r'',
             r'']
mov_means_l = [52,
               52]
               #52,
               #52,
               #52,
               #31,
               #31,
               #31,
               #31,
               #31]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [14]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells001\preprocessing\20230923_PL35_wells001_Tiara_done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells101\preprocessing\20230923_PL35_wells101_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells201\preprocessing\20230923_PL35_wells201_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells301\preprocessing\20230923_PL35_wells301_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells401\preprocessing\20230923_PL35_wells401_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells501\preprocessing\20230923_PL35_wells501_Yael_Done_qc_doneqc']
mmap_dir_l = [r'']
mov_means_l = [31]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [11]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\nasquatch\data\2p\peter\slice\20230830_SL1PL28\slice\preprocessing\20230830_SL1PL28_slice_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230830_SL1PL28\wells001\preprocessing\20230830_SL1PL28_wells001_Done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells001\preprocessing\20230915_PL52_wells001_Yael_Done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells101\preprocessing\20230915_PL52_wells101_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells201\preprocessing\20230915_PL52_wells201_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells301\preprocessing\20230915_PL52_wells301_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells401\preprocessing\20230915_PL52_wells401_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL48\wells001\preprocessing\20230916_PL48_wells001_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL48\wells101\preprocessing\20230916_PL48_wells101_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230920_PL58\wells001\preprocessing\20230920_PL58_wells001_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230920_PL58\wells101\preprocessing\20230920_PL58_wells101_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230920_PL58\wells201\preprocessing\20230920_PL58_wells201_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230920_PL58\wells301\preprocessing\20230920_PL58_wells301_Done_qc_doneqc']
mmap_dir_l = [r'',
             r'',
             r'']
mov_means_l = [52,
               52,
               31]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [8]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\nasquatch\data\2p\peter\in_vivo\220406_PK24PL10\mouse\preprocessing\220406_PK24PL10_mouse_jordan_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\220406_PK24PL10\wells101\preprocessing\220406_PK24PL10_mouse_jordan_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\220427_PK27PL20\mouse\preprocessing\220427_PK27PL20_mouse_Tiara_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\220427_PK27PL20\wells101\preprocessing\220427_PK27PL20_wells101_Tiara_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\221018_PS10PL24\mouse\preprocessing\221018_PS10PL24_mouse_Yael_Done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\221018_PS10PL24\wells101\preprocessing\221018_PS10PL24_wells101_Yael_Done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\in_vivo\230519_DG41PL52\mouse\preprocessing\230519_DG41PL52_mouse_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230519_DG41PL52\wells101\preprocessing\230519_DG41PL52_wells101_Done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\230525_DG42PL39\mouse\preprocessing\230525_DG42PL39_mouse_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230525_DG42PL39\wells101\preprocessing\230525_DG42PL39_wells101_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230526_PK84PL50\mouse\preprocessing\230526_PK84PL50_mouse_Yael_Done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\230526_PK84PL50\wells101\preprocessing\230526_PK84PL50_wells101_Yael_Done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\230617_DG70PL54\mouse\preprocessing\230617_DG70PL54_mouse_Yael_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230617_DG70PL54\wells101\preprocessing\230617_DG70PL54_wells101_Yael_done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\230720_PK97PL48\mouse\preprocessing\230720_PK97PL48_mouse_Yael_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230720_PK97PL48\wells101\preprocessing\230720_PK97PL48_wells101_Yael_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230726_DG47PL54\mouse\preprocessing\230726_DG47PL54_mouse_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230726_DG47PL54\wells101\preprocessing\230726_DG47PL54_wells101_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230802_DG48PL58\mouse\preprocessing\230802_DG48PL58_mouse_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230802_DG48PL58\wells101\preprocessing\230802_DG48PL58_wells101_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230807_DG84PL52\mouse\preprocessing\230807_DG84PL52_mouse_Yael_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230807_DG84PL52\wells101\preprocessing\230807_DG84PL52_wells101_Yael_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230810_DG87PL55\mouse\preprocessing\230810_DG87PL55_mouse_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\230810_DG87PL55\wells101\preprocessing\230810_DG87PL55_wells101_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\230905_PS56PL58\mouse\preprocessing\230905_PS56PL58_mouse_maddy_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230905_PS56PL58\wells101\preprocessing\230905_PS56PL58_wells101_maddy_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\231030_DG60PL67\mouse\preprocessing\231030_DG60PL67_mouse_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\231030_DG60PL67\wells\preprocessing\231030_DG60PL67_wells_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\dipping\20230823_PL28\wells001\preprocessing\20230823_PL28_wells001_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230823_PL28\wells101\preprocessing\20230823_PL28_wells101_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230823_PL28\wells201\preprocessing\20230823_PL28_wells201_Tiara_done_qc_doneqc']
mmap_dir_l = [r'',
             r'',
             r'',
             r'',
             r'',
             r'']
mov_means_l = [52,
               52,
               52,
               52,
               52,
               52]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [5]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\nasquatch\data\2p\peter\slice\20230831_SL1PL55\slice\preprocessing\20230831_SL1PL55_slice_Yael_Done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230831_SL1PL55\wells001\preprocessing\20230831_SL1PL55_wells001_Yael_Done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\slice\20230822_SL2PL52\slice\preprocessing\20230822_SL2PL52_slice_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230822_SL2PL52\wells001\preprocessing\20230822_SL2PL52_wells001_maddy_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\slice\20230801_SL1PL35\slice\preprocessing\20230801_SL1PL35_slice_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230801_SL1PL35\wells101\preprocessing\20230801_SL1PL35_wells101_maddy_done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\slice\20230607_SL1PL47\slice\preprocessing\20230607_SL1PL47_slice_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\slice\20230607_SL1PL47\wells001\preprocessing\20230607_SL1PL47_wells001_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\slice\20230509_DA1PL53\slice\preprocessing\20230509_DA1PL53_slice_Yael_Done_qc_done',
             #r'\\nasquatch\data\2p\peter\slice\20230509_DA1PL53\wells101\preprocessing\20230509_DA1PL53_wells101_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells001\preprocessing\20230916_PL54_wells001_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells101\preprocessing\20230916_PL54_wells101_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells201\preprocessing\20230916_PL54_wells201_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells301\preprocessing\20230916_PL54_wells301_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells401\preprocessing\20230916_PL54_wells401_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells501\preprocessing\20230916_PL54_wells501_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230902_DG90PL61\mouse\preprocessing\230902_DG90PL61_mouse_maddy_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230902_DG90PL61\wells101\preprocessing\230902_DG90PL61_wells101_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230707_DG72PL32\mouse\preprocessing\230707_DG72PL32_mouse_charlotte_done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\230707_DG72PL32\wells101\preprocessing\230707_DG72PL32_wells101_jordan_done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\220701_PK49PL18\mouse\preprocessing\220701_PK49PL18_mouse_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220701_PK49PL18\wells101\preprocessing\220701_PK49PL18_wells101_jordan_done_qc_doneqc']
mmap_dir_l = [r'',
             r'',
             r'',
             r'',
             r'',
             r'']
mov_means_l = [52,
               52,
               52,
               52,
               52,
               52]
               #52,
               #52,
               #52,
               #52,
               #31,
               #31,
               #31,
               #31,
               #31,
               #31,
               #52,
               #52,
               #52,
               #52,
               #52,
               #52]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [2]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\nasquatch\data\2p\peter\slice\20230616_SL1PL28\slice\preprocessing\20230616_SL1PL28_slice_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230616_SL1PL28\wells001\preprocessing\20230616_SL1PL28_wells001_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230606_SL1PL35\slice\preprocessing\20230606_SL1PL35_slice_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230606_SL1PL35\washes\preprocessing\20230606_SL1PL35_washes_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\preprocessing\231108_DG61PL62_mouse_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\wells\preprocessing\231108_DG61PL62_wells_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231105_DG39PL32\mouse\preprocessing\231105_DG39PL32_mouse_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231105_DG39PL32\wells\preprocessing\231105_DG39PL32_wells_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231031_DG53PL28\mouse\preprocessing\231031_DG53PL28_mouse_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231031_DG53PL28\wells\preprocessing\231031_DG53PL28_wells_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\230909_DG88PL58\mouse\preprocessing\230909_DG88PL58_mouse_Yael_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\230909_DG88PL58\wells\preprocessing\230909_DG88PL58_wells_jordan_done_qc_done']
mmap_dir_l = [r'\\nasquatch\data\2p\peter\slice\20230616_SL1PL28\SL1PL28_230616_001_wch0_aniso4_scale4.4AFS2PL4_toref18_wxy.npy',
             r'\\nasquatch\data\2p\peter\slice\20230616_SL1PL28\PL28_230616_001_full_wch0_aniso4_scale4.4AFS2PL4_toref11_wxy.npy',
             r'\\nasquatch\data\2p\peter\slice\20230606_SL1PL35\SL1PL35_230606_001_wch0_aniso4_scale4.4AFS2PL4_toref18_wxy.npy',
             r'\\nasquatch\data\2p\peter\slice\20230606_SL1PL35\PL35_230606_001_wch0_aniso4_scale4.4AFS2PL4_toref9_wxy.npy',
             r'\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\DG61PL62_231108_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy.npy',
             r'\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\DG61PL62_231108_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
             r'\\nasquatch\data\2p\peter\in_vivo\231105_DG39PL32\DG39PL32_231105_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy.npy',
             r'\\nasquatch\data\2p\peter\in_vivo\231105_DG39PL32\DG39PL32_231105_201_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
             r'\\nasquatch\data\2p\peter\in_vivo\231031_DG53PL28\DG53PL28_231031_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy.npy',
             r'\\nasquatch\data\2p\peter\in_vivo\231031_DG53PL28\DG53PL28_231031_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
             r'\\nasquatch\data\2p\peter\in_vivo\230909_DG88PL58\DG88PL58_230909_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref71_wxy.npy',
             r'\\nasquatch\data\2p\peter\in_vivo\230909_DG88PL58\DG88PL58_230909_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy']
mov_means_l = [52,
               52,
               52,
               52,
               52,
               52,
               52,
               52,
               52,
               52,
               52,
               31]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [26]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells601\preprocessing\20230922_PL28_wells601_done_qc_done']
mmap_dir_l = [r'D:\pkalugin\memmap_cache\PL28_230922_601_full_wch1_aniso4_scale4AFS2PL4_toref15_wxy_nobg']
mov_means_l = [31]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [23]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells401\preprocessing\20230922_PL28_wells401_Tiara_done_qc_done']
mmap_dir_l = [r'D:\pkalugin\memmap_cache\PL28_230922_401_full_wch1_aniso4_scale4AFS2PL4_toref18_wxy_nobg']
mov_means_l = [31]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [19]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\nasquatch\data\2p\peter\in_vivo\220408_PK27PL20\mouse\preprocessing\220408_PK27PL20_mouse_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\220408_PK27PL20\wells101\preprocessing\220408_PK27PL20_wells101_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230829_SL1PL56\slice\preprocessing\20230829_SL1PL56_slice_Tiara_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230829_SL1PL56\wells001\preprocessing\20230829_SL1PL56_wells001_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230817_SL1PL55\slice\preprocessing\20230817_SL1PL55_slice_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230817_SL1PL55\wells001\preprocessing\20230817_SL1PL55_wells001_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230817_SL1PL55\wells101\preprocessing\20230817_SL1PL55_wells101_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
             r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells201\preprocessing\20230922_PL28_wells201_maddy_done_qc_done']
mmap_dir_l = [r'D:\pkalugin\memmap_cache\PK27PL20_220408_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PK27PL20_220408_101_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy_nobg',
             r'D:\pkalugin\memmap_cache\SL1PL56_230829_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL56_230829_001_full_wch1_aniso4_scale4AFS2PL4_toref52_wxy_nobg',
             r'D:\pkalugin\memmap_cache\SL1PL55_230817_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref27_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL55_230817_001_full_wch1_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL55_230817_101_full_wch1_aniso4_scale4AFS2PL4_toref39_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230922_001_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230922_201_full_wch1_aniso4_scale4AFS2PL4_toref19_wxy_nobg']
mov_means_l = [52,
              52,
              52,
              52,
              52,
              52,
              52,
              31,
              31]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [16]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [#r'\\nasquatch\data\2p\peter\in_vivo\231106_DG52PL68\mouse\preprocessing\231106_DG52PL68_mouse_jordan_done_qc_peter_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231106_DG52PL68\wells\preprocessing\231106_DG52PL68_wells_jordan_done_qc_peter_done']
mmap_dir_l = [#r'D:\pkalugin\memmap_cache\DG52PL68_231106_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG52PL68_231106_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy_nobg']
mov_means_l = [#52,
              52]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [9]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\nasquatch\data\2p\peter\in_vivo\231105_DG39PL32\mouse\preprocessing\231105_DG39PL32_mouse_maddy_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231105_DG39PL32\wells\preprocessing\231105_DG39PL32_wells_maddy_done']
mmap_dir_l = [r'D:\pkalugin\memmap_cache\DG39PL32_231105_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG39PL32_231105_201_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy_nobg']
mov_means_l = [52,
              52]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [3]:
for m in trange(0,len(data_dir_l)):
    data_dir = data_dir_l[m]
    mmap_dir = mmap_dir_l[m]
    mov_means = mov_means_l[m]
    
    data_dir = data_dir.replace(f'\\',f'/')
    mmap_dir = mmap_dir.replace(f'\\',f'/')
    
    ## Loading all the inputs
    os.chdir(data_dir)
    #cnmf_path = caiman_datadir()+'/example_movies/demoMovie3DYxxbnobg_20240318170305_cnmf.hdf5'
    cnmf_path = os.path.join(data_dir, 'ch0_means_movie_nobg_cnmf.hdf5')
    
    # CNMFE model
    cnmf_model = cnmf.load_CNMF(cnmf_path, 
                                n_processes=1,
                                dview=None)
    #print(f"Successfully loaded CNMF model")
    
    mc_memmapped_fname = [i for i in os.listdir() if 'memmap__' in i][0]
    Yr, dims, T = cm.load_memmap(mc_memmapped_fname)
    #images = np.array(np.reshape(Yr.T, [T] + list(dims), order='F')) 
    images = np.reshape(Yr.T, [T] + list(dims), order='F')
    #print(f"Successfully loaded data")
    
    #d = cnmf_model.estimates.A.shape[0]
    #dims = cnmf_model.estimates.dims
    #axis = 2
    #order = list(range(4))
    #order.insert(0, order.pop(axis))
    #index_permut = np.reshape(np.arange(d), dims, order='F').transpose(
    #        order[:-1]).reshape(d, order='F')
    #A = csc_matrix(cnmf_model.estimates.A)[index_permut, :]
    #dims = tuple(np.array(dims)[order[:3]])
    #d1, d2, d3 = dims
    #nr, T = cnmf_model.estimates.C.shape
    #image_cells = np.array(A.mean(axis=1)).reshape(dims, order='F')
    #coors = get_contours(A, dims, thr=Cthr)
    coors = load_pickle(os.path.join(data_dir, 'ch0_means_movie_nobg_coors.pickle'))
    
    cc = [[l for l in n['coordinates']] for n in coors] # x,y values of contour coordinates for each component
    cc1 = [[(l[:, 0]) for l in n['coordinates']] for n in coors] # x values of contour coordinates for each component
    cc2 = [[(l[:, 1]) for l in n['coordinates']] for n in coors] # y values of contour coordinates for each component
    length = np.ravel([list(map(len, cc)) for cc in cc1])
    shapes = [[np.vstack([np.append(i,np.flip(pt)) for pt in cc[j][i]]) for i in range(len(cc[j]))] for j in range(len(cc))]
    
    # Line up all static inputs
    SNRs = cnmf_model.estimates.SNR_comp
    SNR_min = cnmf_model.estimates.SNRmin
    SOL = np.argsort(-SNRs)
    spcomps = np.reshape(cnmf_model.estimates.A.toarray(),cnmf_model.estimates.dims + (-1,),order='F')
    spcomps = spcomps.transpose([3,2,0,1])
    images2 = images.transpose([0,3,1,2])
    #SOL = np.argsort(-cnmf_model.estimates.SNR_comp) 
    C = cnmf_model.estimates.C
    CY = cnmf_model.estimates.C + cnmf_model.estimates.YrA # temporal loadings
    R = cnmf_model.estimates.Craw # masks applied to raw movie
    
    CYsav = cnmf_model.estimates.CYsav # smoothened CY curve
    def sav_calc(sraw):
        return sg.savgol_filter(sraw,3,1)
    Rsav = np.zeros(R.shape)
    for i in range(R.shape[0]):
        Rsav[i,:] = sav_calc(R[i,:])
    CYsavsort = np.sort(CYsav,axis=1)
    CYsavb10 = np.mean(CYsavsort[:,:int(np.ceil(CYsavsort.shape[1]/10))],axis=1) # bottom 10% mean
    Rsavsort = np.sort(Rsav,axis=1)
    Rsavb10 = np.mean(Rsavsort[:,:int(np.ceil(Rsavsort.shape[1]/10))],axis=1) # bottom 10% mean
    n_range = cnmf_model.estimates.n_range
    if n_range is None:
        CYf = CYsavb10
        Rf = Rsavb10
    else:
        CYf = np.mean(CY[:,n_range],axis=1)
        Rf = np.mean(R[:,n_range],axis=1)
    
    Cn = cnmf_model.estimates.Cn # correlation image (not necessary)
    keepargs = cnmf_model.estimates.keepargs
    SOL = np.array([x for x in list(SOL) if x in list(keepargs)]) # initial search order list
    
    A1 = csc_matrix(cnmf_model.estimates.A)
    nr = A1.shape[1]
    A_corr = scipy.sparse.triu(A1.T * A1)
    A_corr.setdiag(0)
    A_corr = A_corr.tocsc()
    C_corr = scipy.sparse.lil_matrix(A_corr.shape)
    for ii in range(nr):
        overlap_indices = scipy.sparse.find(A_corr[ii, :])[1][scipy.sparse.find(A_corr[ii, :])[2]>Athresh]
        if len(overlap_indices) > 0:
                # we chesk the correlation of the calcium traces for each overlapping components
            corr_values = [scipy.stats.pearsonr(C[ii, :], C[jj, :])[
                0] for jj in overlap_indices]
            C_corr[ii, overlap_indices] = corr_values
    C_tot = C_corr + C_corr.T
    CYR_corr = np.zeros(nr)
    for ii in range(nr):
        CYR_corr[ii] = scipy.stats.pearsonr(R[ii,:],CY[ii,:])[0]
    
    # Initialize all running variables in a single dictionary - just lists of arguments/component IDs (SOL, CSL, CMG, CKG, saved merge groups, trash)
    # check if saved file exists - load that if it does, else instantiate new vars_dict!
    save_path = 'ch0_means_movie_nobg_compfilt.pickle'
    vars_dict1 = load_pickle(save_path)
    
    #%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
    n_processes = n_cores
    if 'dview' in locals():
        cm.stop_server(dview=dview)
    c, dview, n_processes = cm.cluster.setup_cluster(
        backend='multiprocessing', n_processes=n_processes, single_thread=False)
    
    parameter_dict = {#'fnames': cnmf_model.params.data['fnames'][0].decode('UTF-8'),
                      #'dims': cnmf_model.params.data['dims'],
                      'p': cnmf_model.params.temporal['p'],
                      'nb': cnmf_model.params.temporal['nb'],
                      'rf': cnmf_model.params.patch['rf'],
                      'K': cnmf_model.params.init['K'], 
                      'gSig': cnmf_model.params.init['gSig'],
                      'gSiz': cnmf_model.params.init['gSiz'],
                      'stride': cnmf_model.params.patch['stride'],
                      'method_init': cnmf_model.params.init['method_init'],
                      'rolling_sum': cnmf_model.params.init['rolling_sum'], #important
                      'rolling_length': cnmf_model.params.init['rolling_length'],
                      'only_init': cnmf_model.params.patch['only_init'], #also keep an eye on this
                      'ssub': cnmf_model.params.init['ssub'],
                      'tsub': cnmf_model.params.init['tsub'],
                      'merge_thr': cnmf_model.params.merging['merge_thr'], 
                      'bas_nonneg': cnmf_model.params.temporal['bas_nonneg'],
                      'kernel': cnmf_model.params.init['kernel'],
                      'nIter': cnmf_model.params.init['nIter']
                     }
    
    parameters = params.CNMFParams(params_dict=parameter_dict) # CNMFParams is the parameters class
    
    cnmf_model_merged = cnmf.CNMF(n_processes, 
                           params=parameters, 
                           dview=dview)
    cnmf_model_merged.params.data['dims'] = cnmf_model.params.data['dims']
    cnmf_model_merged.params.spatial['n_pixels_per_process'] = cnmf_model.params.spatial['n_pixels_per_process']
    cnmf_model_merged.params.preprocess['n_pixels_per_process'] = cnmf_model.params.preprocess['n_pixels_per_process']
    cnmf_model_merged.params.spatial['nrgthr'] = nrg_thr
    cnmf_model_merged.mmap_file = cnmf_model.mmap_file
    cnmf_model_merged.estimates = deepcopy(cnmf_model.estimates)
    
    cnmf_model_merged.estimates.idx_components = flatten_extend(vars_dict1["SMG"])
    cnmf_model_merged.estimates.idx_components_bad = np.setdiff1d(range(cnmf_model.estimates.C.shape[0]),flatten_extend(vars_dict1["SMG"]))
    cnmf_model_merged.estimates.manual_merge(vars_dict1["SMG"],cnmf_model.params)
    cnmf_model_merged.estimates.select_components(use_object=True)

    print(cnmf_model.estimates.C.shape)
    print(cnmf_model_merged.estimates.C.shape)
    
    Yr = np.transpose(np.reshape(images, (T, -1), order='F'))
    Yr.filename = images.filename
    Yr = cnmf_model_merged.preprocess(Yr)
    cnmf_model_merged.update_temporal(Yr, use_init=False)
    cnmf_model_merged.estimates.normalize_components()
    print(cnmf_model_merged.estimates.C.shape)

    save_path_merged = os.path.join(data_dir, 'ch0_means_movie_nobg_cnmf_merged.hdf5')
    cnmf_model_merged.save(save_path_merged)
    print("Saved merged model")
    
    #~2 hrs on colab05, XX on beastmode, 3 hrs on megatron
    cnmf_model_merged_refit = cnmf_model_merged.refit(images)
    print(cnmf_model_merged_refit.estimates.C.shape)
    
    # save merged and refit outputs
    #print("Saving CNMF model...")
    
    save_path_merged_refit = os.path.join(data_dir, 'ch0_means_movie_nobg_cnmf_merged_refit.hdf5')
    cnmf_model_merged_refit.save(save_path_merged_refit)
    print("Saved refit model")

    if doraw:
        # apply merged/refit model to raw movie
        fname_mmap = mmap_dir + f'/' + os.listdir(mmap_dir)[0]
        Yr2, dims2, T2 = cm.load_memmap(fname_mmap)
        images2 = np.reshape(Yr2.T, [T2] + list(dims2), order='F')
        
        K = cnmf_model_merged_refit.estimates.C.shape[0]
        T = cnmf_model_merged_refit.estimates.C.shape[1]*mov_means
        Cin = cm.source_extraction.cnmf.initialization.resize(cnmf_model_merged_refit.estimates.C, [K,T])
        if Cin.shape[1] < images2.shape[0]:
            Cin = np.concatenate((Cin,np.repeat(Cin[...,Cin.shape[1]-1][:,np.newaxis],images2.shape[0]-Cin.shape[1],axis=1)),axis=1,dtype=Cin.dtype)    
        YrAin = cm.source_extraction.cnmf.initialization.resize(cnmf_model_merged_refit.estimates.YrA, [K,T])
        if YrAin.shape[1] < images2.shape[0]:
            YrAin = np.concatenate((YrAin,np.repeat(YrAin[...,YrAin.shape[1]-1][:,np.newaxis],images2.shape[0]-YrAin.shape[1],axis=1)),axis=1,dtype=YrAin.dtype)  
        
        cnmf_model_mr_raw = cnmf.CNMF(n_processes, 
                               params=parameters, 
                               dview=dview)
        cnmf_model_mr_raw.estimates = deepcopy(cnmf_model_merged_refit.estimates)
        cnmf_model_mr_raw.estimates.C = Cin
        cnmf_model_mr_raw.estimates.YrA = YrAin
        cnmf_model_mr_raw.estimates.b = None
        cnmf_model_mr_raw.estimates.f = None
        cnmf_model_mr_raw.mmap_file = cnmf_model.mmap_file
        
        #~2.5 hrs on colab05, XX on beastmode, 5.5 hrs on megatron
        cnmf_model_mr_raw_refit = cnmf_model_mr_raw.refit(images2)
        print(cnmf_model_mr_raw_refit.estimates.C.shape)
        
        # save merged and refit outputs
        #print("Saving CNMF model...")
        save_path_mr_raw = os.path.join(data_dir, 'ch0_means_movie_nobg_cnmf_mr_raw.hdf5')
        save_path_mr_raw_refit = os.path.join(data_dir, 'ch0_means_movie_nobg_cnmf_mr_raw_refit.hdf5')
        cnmf_model_mr_raw.save(save_path_mr_raw)
        cnmf_model_mr_raw_refit.save(save_path_mr_raw_refit)
        
        print("Saved raw/refit models")

# STOP CLUSTER
cm.stop_server(dview=dview)

  0%|          | 0/1 [00:00<?, ?it/s]

D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 114, using nperseg = 114
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


*** Variable lam has not the same number of components as A ***
*** Variable SNR_comp has not the same number of components as A ***
(882, 114)
(161, 114)


D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\sparse\_dia.py:277: RuntimeWarning: divide by zero encountered in remainder
  c = np.arange(num_rows, dtype=np.intc) - (offsets % max_dim)[:, None]


(161, 114)
Saved merged model
(4897272, 161, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(4897272, 161, 10, 55.68981825241154)
(4897272, 161, 0, 508819565.62143785)
(4897272, 161, 11, 9983238.110095773)
(4897272, 161, 1, 6392993.533333624)
(4897272, 161, 2, 6392993.533333624)


D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 114, using nperseg = 114
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


(4897272, 161, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(4897272, 161, 10, 6392993.533333624)
(4897272, 161, 0, 255362900.81350785)
(4897272, 161, 11, 8683906.732632022)
(4897272, 161, 1, 6245266.9160413025)
(4897272, 161, 2, 6245266.9160413025)


D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 114, using nperseg = 114
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\sparse\_dia.py:277: RuntimeWarning: divide by zero encountered in remainder
  c = np.arange(num_rows, dtype=np.intc) - (offsets % max_dim)[:, None]


(161, 114)
Saved refit model
